In [13]:
import keras
from keras.layers import LSTM, Dropout, Dense
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from sklearn.metrics import confusion_matrix

In [14]:
Base = pd.read_csv("Base.csv")
Base['Time'] = pd.to_datetime(Base['Time'])
Base

,Time,Block Height,Hook Load,WOB,Torque,Flow In,Pump Pressure,RPM,ROP,Evento,Pega,Actividad
0,2015-02-01 00:00:57,57.708860,430.624454,1.301011,1.016953,357.482920,2022.031010,68.507219,0.0,Evento 1,0,RIH
1,2015-02-01 00:01:00,55.942886,429.602631,1.165243,0.680712,360.418803,2058.592517,60.428979,0.0,Evento 1,0,RIH
2,2015-02-01 00:01:03,54.164406,428.439948,0.942667,0.394440,365.651801,2097.301313,52.715292,0.0,Evento 1,0,RIH
3,2015-02-01 00:01:06,52.383372,428.350848,0.769924,0.365904,366.095786,2096.310389,46.199610,0.0,Evento 1,0,RIH
4,2015-02-01 00:01:09,50.617904,428.265523,0.516112,0.341655,366.543219,2127.829605,39.726500,0.0,Evento 1,0,RIH
...,...,...,...,...,...,...,...,...,...,...,...,...
179463,2014-04-23 17:01:33,49.594614,306.238681,6.478149,4.480541,72.917473,1455.683618,89.093420,0.0,Evento 16,1,RIH
179464,2014-04-23 17:01:36,49.594614,306.238681,6.478149,4.480541,72.917473,1455.683618,89.093420,0.0,Evento 16,1,RIH
179465,2014-04-23 17:01:39,49.614314,306.256665,6.008588,4.371957,67.291780,1382.719355,87.606353,0.0,Evento 16,1,RIH
179466,2014-04-23 17:01:42,49.614314,306.256665,6.008588,4.371957,67.291780,1382.719355,87.606353,0.0,Evento 16,1,RIH


In [15]:
Base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179468 entries, 0 to 179467
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Time           179468 non-null  datetime64[ns]
 1   Block Height   179468 non-null  float64       
 2   Hook Load      179468 non-null  float64       
 3   WOB            179468 non-null  float64       
 4   Torque         179468 non-null  float64       
 5   Flow In        179468 non-null  float64       
 6   Pump Pressure  179468 non-null  float64       
 7   RPM            179468 non-null  float64       
 8   ROP            179468 non-null  float64       
 9   Evento         179468 non-null  object        
 10  Pega           179468 non-null  int64         
 11  Actividad      179468 non-null  object        
dtypes: datetime64[ns](1), float64(8), int64(1), object(2)
memory usage: 16.4+ MB


In [16]:
encoder = LabelEncoder()
Base["Evento"] = encoder.fit_transform(Base["Evento"])
Base["Actividad"] = encoder.fit_transform(Base["Actividad"])
Base.set_index('Time',inplace=True)

In [17]:
Base

,Block Height,Hook Load,WOB,Torque,Flow In,Pump Pressure,RPM,ROP,Evento,Pega,Actividad
Time,,,,,,,,,,,
2015-02-01 00:00:57,57.708860,430.624454,1.301011,1.016953,357.482920,2022.031010,68.507219,0.0,0,0,1
2015-02-01 00:01:00,55.942886,429.602631,1.165243,0.680712,360.418803,2058.592517,60.428979,0.0,0,0,1
2015-02-01 00:01:03,54.164406,428.439948,0.942667,0.394440,365.651801,2097.301313,52.715292,0.0,0,0,1
2015-02-01 00:01:06,52.383372,428.350848,0.769924,0.365904,366.095786,2096.310389,46.199610,0.0,0,0,1
2015-02-01 00:01:09,50.617904,428.265523,0.516112,0.341655,366.543219,2127.829605,39.726500,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2014-04-23 17:01:33,49.594614,306.238681,6.478149,4.480541,72.917473,1455.683618,89.093420,0.0,7,1,1
2014-04-23 17:01:36,49.594614,306.238681,6.478149,4.480541,72.917473,1455.683618,89.093420,0.0,7,1,1
2014-04-23 17:01:39,49.614314,306.256665,6.008588,4.371957,67.291780,1382.719355,87.606353,0.0,7,1,1


In [18]:
rec_count = len(Base)
rec_count

179468

In [19]:
def generate_data(X, y, sequence_length, step):
    X_local = []
    y_local = []
    for start in range(0, len(Base) - sequence_length, step):
        end = start + sequence_length
        X_local.append(X[start:end])
        y_local.append(y[end-1])
    return np.array(X_local), np.array(y_local)

In [22]:
Base.Pega

Time
2015-02-01 00:00:57    0
2015-02-01 00:01:00    0
2015-02-01 00:01:03    0
2015-02-01 00:01:06    0
2015-02-01 00:01:09    0
                      ..
2014-04-23 17:01:33    1
2014-04-23 17:01:36    1
2014-04-23 17:01:39    1
2014-04-23 17:01:42    1
2014-04-23 17:01:45    1
Name: Pega, Length: 179468, dtype: int64

In [8]:
X_sequence, y = generate_data(Base.drop(columns=["Pega"]).values, Base.Pega,20*10,1)

In [9]:
X_sequence.shape, y.shape

((179268, 200, 10), (179268,))

In [235]:
model = keras.Sequential()
model.add(LSTM(30, input_shape = (200, 10),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(20,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(10))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy"
              , metrics=[keras.metrics.binary_accuracy]
              , optimizer="adam")

model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_34 (LSTM)              (None, 200, 30)           4920      
                                                                 
 dropout_28 (Dropout)        (None, 200, 30)           0         
                                                                 
 lstm_35 (LSTM)              (None, 200, 20)           4080      
                                                                 
 dropout_29 (Dropout)        (None, 200, 20)           0         
                                                                 
 lstm_36 (LSTM)              (None, 10)                1240      
                                                                 
 dense_17 (Dense)            (None, 1)                 11        
                                                                 
Total params: 10,251
Trainable params: 10,251
Non-tra

In [10]:
training_size = int(len(X_sequence) * 0.7)

In [11]:
X_train, y_train = X_sequence[:training_size], y[:training_size]
X_test, y_test = X_sequence[training_size:], y[training_size:]

In [12]:
np.shape(X_sequence)

(179268, 200, 10)

In [238]:
model.fit(X_train, y_train, batch_size=64, epochs=5)

Epoch 1/5
1961/1961 [==============================] - 393s 197ms/step - loss: 0.1557 - binary_accuracy: 0.9497
Epoch 2/5
1961/1961 [==============================] - 402s 205ms/step - loss: 0.1167 - binary_accuracy: 0.9610
Epoch 3/5
1961/1961 [==============================] - 400s 204ms/step - loss: 0.0984 - binary_accuracy: 0.9652
Epoch 4/5
1961/1961 [==============================] - 375s 191ms/step - loss: 0.0853 - binary_accuracy: 0.9710
Epoch 5/5
1961/1961 [==============================] - 368s 188ms/step - loss: 0.0689 - binary_accuracy: 0.9768


In [244]:
X_test

array([[[ 98.33889182, 232.95248162,   0.        , ...,   0.        ,
           3.        ,   0.        ],
        [ 98.53113322, 232.92943634,   0.        , ...,   0.        ,
           3.        ,   0.        ],
        [ 98.75430367, 232.84002503,   0.        , ...,   0.        ,
           3.        ,   0.        ],
        ...,
        [ 18.53110499,  82.13730598,   0.        , ...,   0.        ,
           3.        ,   0.        ],
        [ 16.6165582 ,  86.31416929,   0.        , ...,   0.        ,
           3.        ,   0.        ],
        [ 15.00029126,  91.15996602,   0.        , ...,   0.        ,
           3.        ,   0.        ]],

       [[ 98.53113322, 232.92943634,   0.        , ...,   0.        ,
           3.        ,   0.        ],
        [ 98.75430367, 232.84002503,   0.        , ...,   0.        ,
           3.        ,   0.        ],
        [ 98.98873452, 232.73140723,   0.        , ...,   0.        ,
           3.        ,   0.        ],
        ...,


In [239]:
y_test_prob = model.predict(X_test, verbose=1)

1681/1681 [==============================] - 77s 43ms/step


In [246]:
np.shape(X_test)

(53781, 200, 10)

In [240]:
model.evaluate(X_test, y_test)

1681/1681 [==============================] - 64s 38ms/step - loss: 0.4202 - binary_accuracy: 0.9018


[0.42019903659820557, 0.9018054604530334]

In [241]:
y_test_pred = np.where(y_test_prob > 0.5, 1, 0)

In [242]:
confusion_matrix(y_test, y_test_pred)

array([[46359,  3048],
       [ 2233,  2141]], dtype=int64)

In [243]:
model.save('LSTM_Model.h5')

In [249]:
y_test_pred

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])